In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

df = pd.read_csv('data/train.csv')

df.head(25)

In [ ]:
df.columns

**1. Data Cleaning and Preprocessing:**
So many missing values, we will need to treat each column depending on the nature of the data in each column and the overall context of your project.

In [ ]:
missing_values = df.isnull().sum()
missing_values[missing_values > 0]

In [ ]:
columns_to_drop = ['ID','SSN']
df = df.drop(columns=columns_to_drop) #<--- dropping columns that are not needed
df.columns

In [ ]:
df.select_dtypes('O').info()

In [ ]:
df['Customer_ID']             = df.Customer_ID.apply(lambda x: int(x[4:], 16))
df['Month']                   = pd.to_datetime(df.Month, format='%B').dt.month
df['Age']                     = df['Age'].astype(str).str.replace(r'\D', '', regex=True).astype(int)
df['Annual_Income']           = df['Annual_Income'].str.replace(r'\D', '', regex=True).astype(float)
df['Num_of_Loan']             = df.Num_of_Loan.astype(str).str.replace(r'\D', '', regex=True).astype(int)
df['Num_of_Delayed_Payment']  = df['Num_of_Delayed_Payment'].str.replace(r'\D', '', regex=True).astype(float)
df['Num_Credit_Inquiries']    = df['Num_Credit_Inquiries'].astype(str).str.replace(r'\D', '', regex=True)
df['Num_Credit_Inquiries']    = df['Num_Credit_Inquiries'].replace('', np.nan).astype(float)
df['Changed_Credit_Limit']    = df['Changed_Credit_Limit'].str.replace(r'_', '0').astype(float)
df['Outstanding_Debt']        = df['Outstanding_Debt'].str.replace(r'(\d)_', r'\1', regex=True).astype(float)
df['Amount_invested_monthly'] = df['Amount_invested_monthly'].replace('__10000__', np.nan).astype(float)
df['Monthly_Balance']         = df['Monthly_Balance'].replace('__-333333333333333333333333333__', np.nan).astype(float)

In [ ]:
df_check = df.copy()
df_check.shape

In [ ]:
def text_cleaning(data):
    if data is np.NaN or not isinstance(data, str):
        return data
    else:
        return str(data).strip('_ ,"')

In [ ]:
df = df_check.applymap(text_cleaning).replace(['', 'nan', '!@9#%8', '#F%$D@*&8', 'NaN'], np.NaN)

In [ ]:
missing_values = df.isnull().sum()
missing_values[missing_values > 0]

In [ ]:
import scipy.stats as stats
import numpy as np
import pandas as pd

def FillMissingWithGroupMode(df, group_column, target_column):
    # Function to convert None to NaN and fill NaN with the mode of the group
    def fill_mode_per_group(data, group, column):
        # Replace None with NaN
        data[column] = data[column].replace([None], np.nan)
        # Calculate and fill the mode for each group
        filled_data = data.groupby(group)[column].transform(lambda x: x.fillna(x.mode().iloc[0]))
        return filled_data

    # Display before filling NaN
    print(f'\nBefore filling NaN in {target_column}:')
    print(df[target_column].isna().sum(), "missing values")
    print(df.groupby(group_column)[target_column].apply(list).head())

    # Fill NaN values
    df[target_column] = fill_mode_per_group(df, group_column, target_column)

    # Display after filling NaN
    print(f'\nAfter filling NaN in {target_column}:')
    print(df[target_column].isna().sum(), "missing values")
    print(df.groupby(group_column)[target_column].apply(list).head())


In [ ]:
#<--- Name
FillMissingWithGroupMode(df, 'Customer_ID', 'Name')

In [ ]:
#<--- Name
FillMissingWithGroupMode(df, 'Customer_ID', 'Payment_Behaviour')

In [ ]:
#<--- Credit_Mix
FillMissingWithGroupMode(df, 'Customer_ID', 'Credit_Mix')

In [ ]:
#<--- Occupation
FillMissingWithGroupMode(df, 'Customer_ID', 'Occupation')

In [ ]:
#<--- Type_of_Loan
df['Type_of_Loan'] = df['Type_of_Loan'].apply(lambda x: x.lower().replace('and ', '').replace(', ', ',').strip() if pd.notna(x) else x)
import re
def get_Diff_Values_Colum(df_column, diff_value=[], sep=',', replace=''):
    column = df_column.dropna()
    for i in column:
        if sep not in i and i not in diff_value:
            diff_value.append(i)
        else:
            for data in map(lambda x:x.strip(), re.sub(replace, '', i).split(sep)):
                if not data in diff_value:
                    diff_value.append(data)
    return dict(enumerate(sorted(diff_value)))
df.groupby('Customer_ID')['Type_of_Loan'].value_counts(dropna=False)
df['Type_of_Loan'].replace([np.NaN], 'No Data', inplace=True)
get_Diff_Values_Colum(df['Type_of_Loan'])

In [ ]:
#<---Num_of_Delayed_Payment
percentile_95 = df['Num_of_Delayed_Payment'].quantile(0.95)
df['Num_of_Delayed_Payment'] = df['Num_of_Delayed_Payment'].apply(lambda x: percentile_95 if x > percentile_95 else x)

df['Num_of_Delayed_Payment'] = df.groupby('Customer_ID')['Num_of_Delayed_Payment'].transform(lambda x: x.fillna(x.median()))

overall_median = df['Num_of_Delayed_Payment'].median()
df['Num_of_Delayed_Payment'].fillna(overall_median, inplace=True)


In [ ]:
#<---Num_Credit_Inquiries
percentile_95_inquiries = df['Num_Credit_Inquiries'].quantile(0.95)
df['Num_Credit_Inquiries'] = df['Num_Credit_Inquiries'].apply(lambda x: percentile_95_inquiries if x > percentile_95_inquiries else x)

df['Num_Credit_Inquiries'] = df.groupby('Customer_ID')['Num_Credit_Inquiries'].transform(lambda x: x.fillna(x.median()))

overall_median_inquiries = df['Num_Credit_Inquiries'].median()
df['Num_Credit_Inquiries'].fillna(overall_median_inquiries, inplace=True)

In [ ]:
#<---Credit_History_Age
def convert_to_total_months(age_str):
    if pd.isna(age_str):
        return None
    parts = age_str.split(' ')
    years = int(parts[0]) if parts[0].isdigit() else 0
    months = int(parts[3]) if len(parts) > 3 and parts[3].isdigit() else 0
    return years * 12 + months

df['Credit_History_Age'] = df['Credit_History_Age'].apply(convert_to_total_months)

df['Credit_History_Age'] = df.groupby('Customer_ID')['Credit_History_Age'].transform(lambda x: x.fillna(x.median()))

overall_median_credit_history = df['Credit_History_Age'].median()
df['Credit_History_Age'].fillna(overall_median_credit_history, inplace=True)

In [ ]:
# Group by 'Customer_ID' and fill NaNs with the median per customer
df['Amount_invested_monthly'] = df.groupby('Customer_ID')['Amount_invested_monthly'].transform(lambda x: x.fillna(x.median()))

# In case the entire 'Amount_invested_monthly' for a customer group is NaN, fill with overall median
overall_median_investment = df['Amount_invested_monthly'].median()
df['Amount_invested_monthly'].fillna(overall_median_investment, inplace=True)

In [ ]:
#<---Monthly_Balance
df['Monthly_Balance'] = df.groupby('Customer_ID')['Monthly_Balance'].transform(lambda x: x.fillna(x.median()))

overall_median_balance = df['Monthly_Balance'].median()
df['Monthly_Balance'].fillna(overall_median_balance, inplace=True)


In [ ]:
#<--- Monthly_Inhand_Salary (Each customer had stable income in dataset)
FillMissingWithGroupMode(df, 'Customer_ID', 'Monthly_Inhand_Salary')


In [ ]:
def replace_outlier_ages(group):
    if len(group) > 1:
        mode_age = group.mode()[0]
        group = group.apply(lambda x: x if x == mode_age else np.nan)
    return group

df['Age'] = df.groupby('Customer_ID')['Age'].transform(replace_outlier_ages)

FillMissingWithGroupMode(df, 'Customer_ID', 'Age')

In [ ]:
def replace_outlier_loan(group):
    if len(group) > 1:
        mode_age = group.mode()[0]
        group = group.apply(lambda x: x if x == mode_age else np.nan)
    return group

df['Num_of_Loan'] = df.groupby('Customer_ID')['Num_of_Loan'].transform(replace_outlier_loan)

FillMissingWithGroupMode(df, 'Customer_ID', 'Num_of_Loan')


In [ ]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
missing_values = df.isnull().sum()
missing_values[missing_values > 0] #<--- missing values are from test dataset which we merged with train dataset
# I realized that the Credit_Score column values from test set were NaN, so I will split the data into train and test sets based on the Credit_Score column

**2. Exploratory Data Analysis (EDA) and Handling Extreme Outliers and error**

In [ ]:
import pandas as pd
import numpy as np

# Assuming 'df' is your DataFrame

def detect_outliers(dataframe):
    outlier_indices_dict = {}

    # Loop over each column in the DataFrame
    for column in dataframe.select_dtypes(include=[np.number]).columns:
        # Calculate Q1 (25th percentile) and Q3 (75th percentile)
        Q1 = dataframe[column].quantile(0.05)
        Q3 = dataframe[column].quantile(0.95)
        IQR = Q3 - Q1

        # Define bounds for outliers
        lower_bound = Q1 - (1.5 * IQR)
        upper_bound = Q3 + (1.5 * IQR)

        # Find outliers
        outliers = dataframe[(dataframe[column] < lower_bound) | (dataframe[column] > upper_bound)]
        outlier_indices_dict[column] = outliers.index.tolist()

    return outlier_indices_dict

# Run the function to detect outliers in all numerical columns
outliers_dict = detect_outliers(df)

# Example to print the outliers for a column
for column, indices in outliers_dict.items():
    print(f"Outliers in column {column}: {len(indices)}")

In [ ]:
# Pre-calculate the typical income for each customer
typical_incomes = df.groupby('Customer_ID')['Annual_Income'].median()

# Merge the median income back to the original dataframe
df = df.join(typical_incomes.rename('Median_Income'), on='Customer_ID')

# Calculate the threshold for being considered an outlier
threshold = 0.5  # You can adjust this threshold as needed
df['Income_Lower_Threshold'] = df['Median_Income'] * (1 - threshold)
df['Income_Upper_Threshold'] = df['Median_Income'] * (1 + threshold)

# Replace outliers with the median income
outlier_condition = (
    (df['Annual_Income'] < df['Income_Lower_Threshold']) |
    (df['Annual_Income'] > df['Income_Upper_Threshold'])
)
df.loc[outlier_condition, 'Annual_Income'] = df.loc[outlier_condition, 'Median_Income']

# Drop the extra columns if you no longer need them
df.drop(['Median_Income', 'Income_Lower_Threshold', 'Income_Upper_Threshold'], axis=1, inplace=True)


In [ ]:
upper_limit = df['Annual_Income'].quantile(0.95)  # Extreme outliers

df['Annual_Income'] = df['Annual_Income'].apply(lambda x: min(x, upper_limit))

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.hist(df['Annual_Income'], bins=50, color='blue', edgecolor='black')  # Reduced number of bins for clarity
plt.title('Capped Distribution of Annual Income')
plt.xlabel('Annual Income')
plt.ylabel('Frequency')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(9, 3))
plt.hist(df['Age'], bins=60, color='blue', edgecolor='black')
plt.title('Distribution of Age')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Define the upper and lower limits
upper_limit = df['Num_Bank_Accounts'].quantile(0.95)  # For extreme outliers
lower_limit = 0  # Minimum realistic value

df['Num_Bank_Accounts'] = df['Num_Bank_Accounts'].apply(lambda x: min(max(x, lower_limit), upper_limit))

plt.figure(figsize=(9, 3))
plt.hist(df['Num_Bank_Accounts'], bins=40, color='blue', edgecolor='black')
plt.title('Distribution of Num_Bank_Accounts')
plt.xlabel('Num_Bank_Accounts')
plt.ylabel('Frequency')
plt.show()


In [ ]:
import seaborn as sns
plt.figure(figsize = (6, 4))
sns.boxplot(data = df,  x = 'Credit_Score', y = 'Outstanding_Debt')
plt.show()
#<--- more debt leads to lower credit score

In [ ]:
plt.figure(figsize = (6, 3))
sns.boxplot(data = df,  x = 'Credit_Score', y = 'Monthly_Balance')
plt.show()

#<--- more balance leads to higher credit score

plt.figure(figsize = (20, 12))
sns.barplot(data = df, x = 'Occupation', y = 'Annual_Income')
plt.show()

#<--- this looks like not a realistic data

In [ ]:
import matplotlib.pyplot as plt

# Define the upper and lower limits
upper_limit = df['Num_Credit_Card'].quantile(0.95)  # For extreme outliers
lower_limit = 0  # Minimum realistic value

df['Num_Credit_Card'] = df['Num_Credit_Card'].apply(lambda x: min(max(x, lower_limit), upper_limit))

plt.figure(figsize=(10, 4))
plt.hist(df['Num_Credit_Card'], bins=40, color='blue', edgecolor='black')
plt.title('Distribution of Num_Credit_Card')
plt.xlabel('Num_Credit_Card')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.figure(figsize = (6, 4))
sns.boxplot(data = df,  x = 'Credit_Score', y = 'Num_Credit_Card')
plt.show()

In [ ]:
plt.figure(figsize = (6, 4))
sns.boxplot(data = df,  x = 'Credit_Score', y = 'Num_of_Loan')
plt.show()

In [ ]:
upper_limit = df['Interest_Rate'].quantile(0.95)  # Extreme outliers

df['Interest_Rate'] = df['Interest_Rate'].apply(lambda x: min(x, upper_limit))

plt.figure(figsize=(10, 4))
plt.hist(df['Interest_Rate'], bins=11, color='blue', edgecolor='black')
plt.title('Distribution of Interest_Rate')
plt.xlabel('Interest_Rate')
plt.ylabel('Frequency')
plt.show()

In [ ]:
upper_limit = df['Total_EMI_per_month'].quantile(0.95)  # Extreme outliers

df['Total_EMI_per_month'] = df['Total_EMI_per_month'].apply(lambda x: min(x, upper_limit))

plt.figure(figsize=(10, 4))
plt.hist(df['Total_EMI_per_month'], bins=11, color='blue', edgecolor='black')
plt.title('Distribution of Total_EMI_per_month')
plt.xlabel('Total_EMI_per_month')
plt.ylabel('Frequency')
plt.show()

In [ ]:
upper_limit = df['Amount_invested_monthly'].quantile(0.95)  # Extreme outliers

df['Amount_invested_monthly'] = df['Amount_invested_monthly'].apply(lambda x: min(x, upper_limit))

plt.figure(figsize=(10, 4))
plt.hist(df['Amount_invested_monthly'], bins=11, color='blue', edgecolor='black')
plt.title('Distribution of Amount_invested_monthly')
plt.xlabel('Amount_invested_monthly')
plt.ylabel('Frequency')
plt.show()

In [ ]:
import seaborn as sns
plt.figure(figsize = (20, 20))
sns.countplot(data = df, x = 'Occupation', hue = 'Payment_of_Min_Amount')

**3. Feature Engineering:**

In [ ]:
categorical_columns =df.select_dtypes('O').columns
display(categorical_columns)

In [ ]:
categorical_columns_head = df.select_dtypes('O').head(5)
categorical_columns_head

In [ ]:
for col in categorical_columns:
    print(f"\nColumn: {col}")
    print("Unique values count:", df[col].nunique())
    print("Value counts:")
    #print(df[col].value_counts(dropna=False).head(1))

In [ ]:
df.drop('Name', axis=1, inplace=True) #<--- dropping 'Name' column as it has too many unique values
df = pd.get_dummies(df, columns=['Occupation', 'Credit_Mix', 'Payment_of_Min_Amount', 'Payment_Behaviour'], drop_first=True)

df.columns


In [ ]:
print(df['Type_of_Loan'].value_counts(dropna=False).head(7))

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

df['Type_of_Loan'] = df['Type_of_Loan'].str.split(',')

mlb = MultiLabelBinarizer()

type_of_loan_mlb = pd.DataFrame(mlb.fit_transform(df['Type_of_Loan']), columns=mlb.classes_, index=df.index)
df = df.join(type_of_loan_mlb)

df.drop('Type_of_Loan', axis=1, inplace=True)

df.head()


In [ ]:
numerical_columns = df.select_dtypes('number').columns
display(numerical_columns)

In [ ]:
numerical_columns_head = df.select_dtypes('number').head(10)
numerical_columns_head

In [ ]:
agg_functions = ['mean', 'median', 'max', 'min', 'std']
aggregated_df = df.groupby('Customer_ID').agg({
    'Annual_Income': agg_functions,
    'Num_of_Loan': agg_functions,
    'Outstanding_Debt': agg_functions,
}).reset_index()

# Rename columns except for 'Customer_ID'
aggregated_df.columns = ['Customer_ID'] + ['{}_{}'.format(col[0], col[1]) for col in aggregated_df.columns[1:]]

# Merge the DataFrames
df = df.merge(aggregated_df, on='Customer_ID', how='left')
df.head()

In [ ]:
# Creating a new feature as the ratio of Outstanding_Debt to Annual_Income
df['Debt_Income_Ratio'] = df['Outstanding_Debt'] / df['Annual_Income']

# Calculate the ratio of Monthly_Balance to Monthly_Inhand_Salary
df['Balance_Salary_Ratio'] = df['Monthly_Balance'] / df['Monthly_Inhand_Salary']

# Calculate the ratio of Total_EMI_per_month to Monthly_Inhand_Salary
df['EMI_Salary_Ratio'] = df['Total_EMI_per_month'] / df['Monthly_Inhand_Salary']

df.isna().sum()

In [ ]:
def credit_utilization_category(ratio):
    if ratio < 30:
        return 'Low'
    elif ratio < 60:
        return 'Medium'
    else:
        return 'High'

# Apply the function to create a new categorical feature
df['Credit_Utilization_Category'] = df['Credit_Utilization_Ratio'].apply(credit_utilization_category)

# One-hot encoding of the new categorical feature
df = pd.get_dummies(df, columns=['Credit_Utilization_Category'], drop_first=True)



**4. Splitting the data: reminder that original test.csv target variable was already dropped**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Dropping 'Customer_ID' and 'Month' from the dataframe
df = df.drop(columns=['Customer_ID', 'Month', 'Annual_Income', 'Num_of_Loan', 'Outstanding_Debt'], errors='ignore')

# Splitting the data into features (X) and target (y)
X = df.drop(columns=['Credit_Score'])
y = df['Credit_Score']

# Splitting the dataset into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print("inf in X_train:", X_train.isin([np.inf, -np.inf]).sum().sum())

import numpy as np

# Identify columns with infinite values
inf_columns = X_train.columns[X_train.isin([np.inf, -np.inf]).any()].tolist()

print("Columns with Infinite Values:", inf_columns)



**5. Feature Selection: Correlation, RandomForest Importance, RFECV**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Assuming X_train and y_train are already defined

# 1. Correlation Analysis on Training Data
corr_matrix = X_train.corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
plt.figure(figsize=(16, 12))
sns.heatmap(corr_matrix, mask=mask, cmap='coolwarm', vmax=.3, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.title("Correlation Matrix for Training Data")
plt.show()

# Identify highly correlated variables (0.85 threshold)
high_corr_var = [col for col in corr_matrix.columns if any(corr_matrix[col] > 0.85)]
print("Highly correlated variables:", high_corr_var)

# 2. Feature Importance with Random Forest on Training Data
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
feature_importances = pd.DataFrame(rf.feature_importances_, index=X_train.columns, columns=['importance']).sort_values('importance', ascending=False)
plt.figure(figsize=(12, 10))
sns.barplot(x=feature_importances['importance'], y=feature_importances.index)
plt.title("Feature Importances in Training Data")
plt.show()

# 3. Recursive Feature Elimination with Cross-Validation (RFECV) on Training Data
rfecv = RFECV(estimator=rf, step=5, cv=StratifiedKFold(5), scoring='accuracy', n_jobs=-1)
rfecv.fit(X_train, y_train)
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(rfecv.cv_results_['mean_test_score']) + 1), rfecv.cv_results_['mean_test_score'], marker='o')
plt.title("RFECV - Number of Features vs. CV Score")
plt.show()

# Final selected features
selected_features = X_train.columns[rfecv.support_]
print("Selected Features:", selected_features)




In [ ]:
import pandas as pd

# Assuming X_train is your DataFrame
corr_matrix = X_train.corr()

# Find pairs of highly correlated features
high_corr_var = np.where(corr_matrix > 0.85)
high_corr_pairs = [(corr_matrix.columns[x], corr_matrix.columns[y])
                   for x, y in zip(*high_corr_var) if x != y and x < y]

for pair in high_corr_pairs:
    print(f"High correlation between {pair[0]} and {pair[1]}: {corr_matrix.loc[pair[0], pair[1]]}")


In [ ]:
# Get the RandomForest model from RFECV (assuming rfecv.estimator_ is a RandomForest model)
rf_model = rfecv.estimator_

# Get feature importances
importances = rf_model.feature_importances_

# Map these importances to the corresponding features
feature_importances = pd.Series(importances, index=X_train.columns[rfecv.support_]).sort_values(ascending=False)

# Select the top 10 features
feature_importances = feature_importances.head(20).index
print("Top 10 features:", feature_importances)

In [ ]:
### High training accuracy but low validation accuracy indicates overfitting
### Let's try to reduce overfitting by tuning the hyperparameters of the Random Forest model


from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import classification_report

#param_grid = {
#    'n_estimators': [50, 100, 200],
#    'max_depth': [None, 10, 20, 30],
#    'min_samples_split': [2, 5, 10],
#    'min_samples_leaf': [1, 2, 4],
#    'max_features': ['sqrt']  ##<--- this is the default value. not sure
#}

param_grid = {
    'n_estimators': [100, 150, 200],
    'max_depth': [10, 15, 20],
    'min_samples_split': [10, 15, 20],
    'min_samples_leaf': [4, 6, 8],
    'max_features': ['sqrt', 'log2'],
}



rf = RandomForestClassifier(random_state=42, n_jobs=-1)

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

grid_search.fit(X_train[selected_features], y_train)

best_rf = grid_search.best_estimator_

#<--- print stuff
y_train_pred = best_rf.predict(X_train[selected_features])
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Training Set Accuracy:", train_accuracy)

y_valid_pred = best_rf.predict(X_valid[selected_features])
valid_accuracy = accuracy_score(y_valid, y_valid_pred)
print("Validation Set Accuracy:", valid_accuracy)

print(classification_report(y_valid, y_valid_pred))

print("Best parameters found: ", grid_search.best_params_)


In [ ]:
#<--- cross validation scores
cross_val_scores = cross_val_score(best_rf, X_train[selected_features], y_train, cv=5, scoring='accuracy')

print("Cross-validation scores:", cross_val_scores)
print("Mean cross-validation score:", cross_val_scores.mean())


In [ ]:
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt
import numpy as np

#<--- learning curve
train_sizes, train_scores, valid_scores = learning_curve(
    estimator=rf,
    X=X_train[selected_features],
    y=y_train,
    train_sizes=np.linspace(0.1, 1.0, 10),
    cv=5,
    n_jobs=-1,
    scoring='accuracy'
)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

valid_mean = np.mean(valid_scores, axis=1)
valid_std = np.std(valid_scores, axis=1)

plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, label='Training score', color='blue', marker='o')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color='blue', alpha=0.15)

plt.plot(train_sizes, valid_mean, label='Cross-validation score', color='green', marker='o')
plt.fill_between(train_sizes, valid_mean - valid_std, valid_mean + valid_std, color='green', alpha=0.15)

plt.title('Learning Curve')
plt.xlabel('Training Data Size')
plt.ylabel('Accuracy')
plt.legend(loc='best')
plt.show()
